# Finding Symbols


Finding the ticker symbol, security identifier, the sector, and other metadata is easy if you know where to look.  This guide is intended to introduce some methods for searching, screening, and discovery.

For maximum coverage and functionality, install OpenBB with `[all]` packages.

The examples here will assume that the OpenBB Platform has been installed, the environment is active, and it has been imported into a Python session.  If the installation is fresh, or an extension was just installed, the Python interface will need to be rebuilt.  It will only take a few moments to complete.

In [1]:
from openbb import obb


The simplest way to find tickers for a company is with a simple fuzzy query.

## Search the SEC

Perform a quick search using the `openbb-sec` provider.

### Find a Company

Use an empty string, `""`, to return the complete list - over 10,000.

In [2]:
all_companies = obb.equity.search("", provider="sec")

len(all_companies.results)


10840

The SEC sorts this list by market cap.  Applying the `to_df()` method to `all_companies` will show Apple on top

In [3]:
all_companies.to_df().head(10)


,symbol,name,cik
0,AAPL,Apple Inc.,320193
1,MSFT,MICROSOFT CORP,789019
2,GOOGL,Alphabet Inc.,1652044
3,AMZN,AMAZON COM INC,1018724
4,NVDA,NVIDIA CORP,1045810
5,META,"Meta Platforms, Inc.",1326801
6,BRK-B,BERKSHIRE HATHAWAY INC,1067983
7,TSLA,"Tesla, Inc.",1318605
8,LLY,ELI LILLY & Co,59478
9,V,VISA INC.,1403161


### Find an Institution

Some reporting companies, like invesment trusts and insurance companies, do not have a ticker symbol directly associated with them.  Filers in the US will have a CIK number, used to retrieve documents from the SEC.

In [4]:
instututions = obb.regulators.sec.institutions_search("Berkshire Hathaway").to_df()
instututions


,name,cik
0,BERKSHIRE HATHAWAY ENERGY CO,0001081316
1,BERKSHIRE HATHAWAY FINANCE CORP,0001274791
2,BERKSHIRE HATHAWAY HOMESTATE INSURANCE CO.,0000829771
3,BERKSHIRE HATHAWAY INC /DE/,0000109694
4,BERKSHIRE HATHAWAY INC/DE,0000109694
5,BERKSHIRE HATHAWAY INC,0001067983
6,BERKSHIRE HATHAWAY LIFE INSURANCE CO OF NEBRASKA,0001015867
7,LMZ & BERKSHIRE HATHAWAY CO,0001652795


### Find a Filing

Search for filings by CIK or ticker symbol.

In [11]:
homestate_filings = obb.equity.fundamental.filings(cik="0000829771", provider="sec")

homestate_filings.to_df().iloc[-1]


type                                                                  13F-NT
link                       https://www.sec.gov/Archives/edgar/data/000082...
report_date                                                       2023-09-30
accepted_date                                      2023-11-14 16:15:06+00:00
act                                                                       34
items                                                                       
primary_doc_description                                                     
primary_doc                                   xslForm13F_X02/primary_doc.xml
accession_number                                        0000950123-23-010929
file_number                                                        028-02226
film_number                                                        231406391
is_inline_xbrl                                                             0
is_xbrl                                                                    0

Or, search by form type.

In [13]:
aapl_filings = obb.equity.fundamental.filings("AAPL", type="4", provider="sec")

aapl_filings.to_df().iloc[-1]


type                                                                       4
link                       https://www.sec.gov/Archives/edgar/data/000032...
report_date                                                       2023-11-10
accepted_date                                      2023-11-14 18:31:09+00:00
act                                                                         
items                                                                       
primary_doc_description                                               FORM 4
primary_doc                               xslF345X05/wk-form4_1700004649.xml
accession_number                                        0000320193-23-000109
file_number                                                                 
film_number                                                                 
is_inline_xbrl                                                             0
is_xbrl                                                                    0

## Screen Markets

Screeners provide a targeted search, a tool for comparison and discovery.  Find stocks from around the world with the screener endpoint, and the `openbb-fmp` provider.

### Find Stocks From India

In [16]:
results = obb.equity.screener(country="IN", provider="fmp").to_df()
display(len(results))
results.head(5).convert_dtypes()


1822

,symbol,name,market_cap,sector,industry,beta,price,last_annual_dividend,volume,exchange,exchange_name,country,is_etf,actively_trading
0,RELIANCE.NS,Reliance Industries Limited,15895961017165,Energy,Oil & Gas Refining & Marketing,0.721409,2349.35,9,2244671,NSE,National Stock Exchange of India,IN,False,True
1,TCS.NS,Tata Consultancy Services Limited,12878392323686,Technology,Information Technology Services,0.528,3519.6,36,1473232,NSE,National Stock Exchange of India,IN,False,True
2,HDFCBANK.NS,HDFC Bank Limited,11345550050918,Financial Services,Banks—Regional,0.714285,1505.2,19,9665132,NSE,National Stock Exchange of India,IN,False,True
3,ICICIBANK.NS,ICICI Bank Limited,6456176076595,Financial Services,Banks—Regional,0.808,921.45,8,10561389,NSE,National Stock Exchange of India,IN,False,True
4,INFY.NS,Infosys Limited,5945857635942,Technology,Information Technology Services,0.596,1436.6,36,3584088,NSE,National Stock Exchange of India,IN,False,True


### Search by Sector

In [17]:
sector_results = obb.equity.screener(country="IN", sector="Financial Services", provider="fmp").to_df()
display(len(sector_results))
sector_results.head(5).convert_dtypes()


190

,symbol,name,market_cap,sector,industry,beta,price,last_annual_dividend,volume,exchange,exchange_name,country,is_etf,actively_trading
0,HDFCBANK.NS,HDFC Bank Limited,11345550050918,Financial Services,Banks—Regional,0.714285,1505.2,19.0,9665132,NSE,National Stock Exchange of India,IN,False,True
1,ICICIBANK.NS,ICICI Bank Limited,6456176076595,Financial Services,Banks—Regional,0.808,921.45,8.0,10561389,NSE,National Stock Exchange of India,IN,False,True
2,SBIN.NS,State Bank of India,5031248625920,Financial Services,Banks—Regional,0.74,563.75,11.3,12710950,NSE,National Stock Exchange of India,IN,False,True
3,BAJFINANCE.NS,Bajaj Finance Limited,4282998873754,Financial Services,Credit Services,0.947,7068.6,30.0,1901753,NSE,National Stock Exchange of India,IN,False,True
4,LICI.NS,Life Insurance Corporation of India,3870900117504,Financial Services,Insurance—Life,<NA>,612.0,3.0,500820,NSE,National Stock Exchange of India,IN,False,True


### Search by Industry

In [20]:
industry_results = obb.equity.screener(country="IN", industry="manufacturing").to_df()
display(len(industry_results))
industry_results.head(5).convert_dtypes()


119

,symbol,name,market_cap,sector,industry,beta,price,last_annual_dividend,volume,exchange,exchange_name,country,is_etf,actively_trading
0,PAGEIND.NS,Page Industries Limited,418555116755,Consumer Cyclical,Apparel Manufacturing,0.462,37525.45,300.0,11199,NSE,National Stock Exchange of India,IN,False,True
1,KPRMILL.NS,K.P.R. Mill Limited,284132900800,Consumer Cyclical,Apparel Manufacturing,0.689,831.25,4.15,342210,NSE,National Stock Exchange of India,IN,False,True
2,ABFRL.NS,Aditya Birla Fashion and Retail Limited,203500974285,Consumer Cyclical,Apparel Manufacturing,0.152,214.4,<NA>,1363689,NSE,National Stock Exchange of India,IN,False,True
3,TRIDENT.NS,Trident Limited,189060118374,Consumer Cyclical,Textile Manufacturing,0.532,37.1,0.36,7863777,NSE,National Stock Exchange of India,IN,False,True
4,WELSPUNIND.NS,Welspun India Limited,157481652339,Consumer Cyclical,Textile Manufacturing,0.423,162.05,0.1,3348644,NSE,National Stock Exchange of India,IN,False,True


### Search by Exchange

Some countries, like America, have multiple exchanges.  Narrow the search by combining two or more parameters.  The example below finds the companies listed on the American Stock Exchange (AMEX) that are domiciled in China.

In [22]:
exchange_results = obb.equity.screener(exchange="amex", country="CN").to_df()
display(len(exchange_results))
exchange_results.convert_dtypes()


4

,symbol,name,market_cap,sector,industry,beta,price,volume,exchange,exchange_name,country,is_etf,actively_trading
0,AMBO,Ambow Education Holding Ltd.,4314134,Consumer Defensive,Education & Training Services,0.448,0.1521,54634,AMEX,American Stock Exchange,CN,False,True
1,ITP,"IT Tech Packaging, Inc.",2922131,Basic Materials,Paper & Paper Products,-0.1,0.2903,17827,AMEX,American Stock Exchange,CN,False,True
2,DXF,Dunxin Financial Holdings Limited,1232011,Financial Services,Credit Services,1.632,0.31,259614,AMEX,American Stock Exchange,CN,False,True
3,CPHI,"China Pharma Holdings, Inc.",1084973,Healthcare,Drug Manufacturers—Specialty & Generic,0.875,0.0949,483393,AMEX,American Stock Exchange,CN,False,True


## Find an Index

List all indices from a source with:

In [23]:
indices = obb.index.available(provider="yfinance").to_df()
display(len(indices))


274

Filter the list down by querying the DataFrame.

In [24]:
indices[indices["name"].str.contains("ASX 200")]


,name,code,symbol
88,S&P/ASX 200 Index (AUD),au_asx200,^AXJO
90,S&P/ASX 200 Energy Sector Index (AUD),au_energy,^AXEJ
91,S&P/ASX 200 Resources Sector Index (AUD),au_resources,^AXJR
92,S&P/ASX 200 Materials Sector Index (AUD),au_materials,^AXMJ
94,S&P/ASX 200 Industrials Sector Index (AUD),au_industrials,^AXNJ
95,S&P/ASX 200 Consumer Discretionary Sector Inde...,au_discretionary,^AXDJ
96,S&P/ASX 200 Consumer Staples Sector Index (AUD),au_staples,^AXSJ
97,S&P/ASX 200 Health Care Sector Index (AUD),au_health,^AXHJ
98,S&P/ASX 200 Financials Sector Index (AUD),au_financials,^AXFJ
99,S&P/ASX 200 A-REIT Industry Index (AUD),au_reit,^AXPJ


With the `openbb-yfinance` extension, index time series can be  loaded using the ticker symbol or short code.  Non-American indices have a code beginning with the two-letter country code.

In [25]:
(
    obb.index.market("au_utilities", provider="yfinance").to_df().tail(1)
    == obb.index.market("^AXUJ", provider="yfinance").to_df().tail(1)
)


,open,high,low,close,volume
date,,,,,
2023-11-17,True,True,True,True,True
